<a href="https://colab.research.google.com/github/SM24-Industrial-Software-Dev/ML-forecasting-NOx-levels/blob/AP-Issue%234-API/Demos/Receive_Conc_API_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
from google.colab import userdata

credentials = ee.ServiceAccountCredentials("yeshiva-summer-2024-1@yu-summer-2024.iam.gserviceaccount.com", key_data=userdata.get('GCP_CREDENTIALS'))
ee.Initialize(credentials = credentials, project='yu-summer-2024', opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
import pandas as pd

def get_Data(locations: ee.FeatureCollection,
             start_date: str,
             end_date: str,
             cloudmasking: float) -> pd.DataFrame:


  adminSelect=locations
  no2Raw = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2')

  CLOUD_MASK_FRACTION = cloudmasking
  def maskClouds(image):
    cf = image.select('cloud_fraction')
    mask=cf.lte(CLOUD_MASK_FRACTION)
    return image.updateMask(mask).copyProperties(image)

  try:
    startDate, endDate = ee.Date(start_date), ee.Date(end_date)
  except:
    print("Invalid dates")
    return None

  ndays = endDate.difference(startDate, 'days')

  def create_date_list_fn(startDate):
    def date_list_fn(days):
      return startDate.advance(days, 'days')
    return date_list_fn

  # generate the list of dates

  date_list_fn = create_date_list_fn(startDate)
  list_of_dates = ee.List.sequence(0, ndays, 1).map(date_list_fn)

  def image_mediancomposite_by_date(date):
    return ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2')\
    .filterDate(ee.Date(date), ee.Date(date).advance(1, 'day'))\
    .map(maskClouds) \
    .select('tropospheric_NO2_column_number_density')\
    .median()\
    .set('system:time_start', ee.Date(date).millis())

  no2 = ee.ImageCollection(
        ee.List.sequence(0, ndays, 1)\
        .map(date_list_fn)\
        .map(image_mediancomposite_by_date)
      )


  def createConc(img):

        def getConc(img):
            no2Mean=img.reduceRegion(
                  reducer = ee.Reducer.mean(),
                  geometry = adminSelect.geometry(),
                  scale =  7000
                ).get('tropospheric_NO2_column_number_density')

            region = img.get('region')

            doy=img.date().getRelative('day', 'year')
            dow=img.date().format('E')
            dt=img.date().format("YYYY-MM-dd")


            # Handle potential missing values
            feature_dict = {
                'DOY': doy,
                'DOW': dow,
                'DATE': dt
            }
            if no2Mean:
                feature_dict['conc'] = no2Mean
                return adminSelect.map(lambda f: f.set(feature_dict)).first()
            else:
                return None

        return getConc(img)

  no2AggChange=no2.filterDate(startDate, endDate) \
                              .map(lambda img:createConc(img)) \
                              .filter(ee.Filter.notNull(['conc']))


  def fc_to_dict(fc):
    prop_names = fc.first().propertyNames()
    prop_lists = fc.reduceColumns(
        reducer=ee.Reducer.toList().repeat(prop_names.size()),
        selectors=prop_names).get('list')

    return ee.Dictionary.fromLists(prop_names, prop_lists)

  NO2_stat_dict = fc_to_dict(no2AggChange).getInfo()
  NO2_stat_df = pd.DataFrame(NO2_stat_dict)

  return NO2_stat_df

In [ ]:
!git clone https://github.com/SM24-Industrial-Software-Dev/ML-forecasting-NOx-levels.git
!pip install import-ipynb
import shutil
# move the file to the working directory
shutil.copy('ML-forecasting-NOx-levels/MSA_class.ipynb', '.')
# This allows us to import the .ipynb flie as a python module
import import_ipynb
from MSA_class import MSA

fatal: destination path 'ML-forecasting-NOx-levels' already exists and is not an empty directory.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.3 MB/s eta 0:00:00
importing Jupyter notebook from MSA_class.ipynb
  Preparing metadata (setup.py) ... done
  Created wheel for pycrs: filename=PyCRS-1.0.2-py3-none-any.whl size=32687 sha256=38674c2d48ef55abda01e5d5885a41013e8ad50ee78d06b3cba69c634ffb2652
  Stored in directory: /root/.cache/pip/wheels/47/1d/70/7a5bdf33347e7c75e95b06b1fa38f076a59a9506653cc24aff
Successfully built pycrs


In [ ]:
msa = MSA()
NY = msa.areas_by_name('New York-Newark-Jersey City, NY-NJ')

In [ ]:
print(get_Data(NY, "2020-03-01", "2020-03-15", 0.3).head())

         ALAND      AWATER CBSAFP CSAFP        DATE  DOW  DOY  GEOID  \
0  15901475257  6489939485  35620   408  2020-03-01  Sun   60  35620   
1  15901475257  6489939485  35620   408  2020-03-02  Mon   61  35620   
2  15901475257  6489939485  35620   408  2020-03-04  Wed   63  35620   
3  15901475257  6489939485  35620   408  2020-03-05  Thu   64  35620   
4  15901475257  6489939485  35620   408  2020-03-07  Sat   66  35620   

          GEOIDFQ LSAD                                NAME  \
0  310M700US35620   M1  New York-Newark-Jersey City, NY-NJ   
1  310M700US35620   M1  New York-Newark-Jersey City, NY-NJ   
2  310M700US35620   M1  New York-Newark-Jersey City, NY-NJ   
3  310M700US35620   M1  New York-Newark-Jersey City, NY-NJ   
4  310M700US35620   M1  New York-Newark-Jersey City, NY-NJ   

                                        NAMELSAD      conc system:index  
0  New York-Newark-Jersey City, NY-NJ Metro Area  0.000034            0  
1  New York-Newark-Jersey City, NY-NJ Metro Ar